In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

from numpy import loadtxt
from sklearn import cluster, datasets, metrics ,preprocessing
from sklearn.metrics import mean_absolute_error,accuracy_score,confusion_matrix
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from IPython.display import clear_output

def try_par(y_true, y_pred):
    minmae=99999
    Rnnn=0
    for nnn in range(800,1200):
        T1 = y_pred*nnn/1000
        mae = mean_absolute_error(y_true,T1)
        if minmae>mae:
            minmae=mae
            Rnnn=nnn
    return Rnnn/1000

def train_model(train_x,train_y,test_x,Re,num_class=3):
    params={'booster':'gbtree',
        'objective': 'multi:softmax', 
        'num_class':num_class,
        'gamma':0.05,
        'max_depth':7,
        'subsample':0.5,
        'colsample_bytree':0.6,
        'silent':1 ,
        'eta': 0.07,
        'seed':1000,
        'nthread':4
       }
    plst = list(params.items())
    num_rounds = 200
    offset = int(len(train_x)*0.8)
    train_x = train_x.reset_index(drop=True)
    train_y = train_y.reset_index(drop=True)
    xgtrain = xgb.DMatrix(train_x.loc[:offset,:], label=train_y[:offset+1])
    xgval = xgb.DMatrix(train_x.loc[offset:,:], label=train_y[offset:])
    watchlist = [(xgtrain, 'train'),(xgval, 'val')]
    model = xgb.train(plst, xgtrain, num_rounds, watchlist,early_stopping_rounds=40)

    test_x = test_x.reset_index(drop=True)
    Re = Re.reset_index(drop=True)
    pred1 = model.predict(xgb.DMatrix(Re),ntree_limit=model.best_iteration)
    pred2 = model.predict(xgb.DMatrix(test_x),ntree_limit=model.best_iteration)
    clear_output()
    return pred1,pred2

rgb_model = XGBRegressor(
                            learning_rate = 0.1,
                            n_estimators = 200,
                            max_depth= 7,
                            objective = 'reg:linear',
                            nthread = 4,
                            seed=1000
                         )
def train_model2(train_x,train_y,train_y2,test_x,Mask1,Mask2):
    rgb_model.fit(train_x[Mask1],train_y2[Mask1])
    T1,T2 = rgb_model.predict(train_x),rgb_model.predict(test_x)
    T1[T1<0]=0
    T2[T2<0]=0
    T3 = train_x['Premium_sum']*T1
    T4 = test_x['Premium_sum']*T2
    A = try_par(train_y[Mask2],T3[Mask2])
    T3,T4=T3*A,T4*A
    return T3,T4,A

def train_model2_1(train_x,train_y,train_y2,test_x,Mask1,Mask2):
    rgb_model.fit(train_x[Mask1],train_y2[Mask1])
    T1,T2 = rgb_model.predict(train_x),rgb_model.predict(test_x)
    T1[T1<0]=0
    T2[T2<0]=0
    T3 = train_x['Premium_sum']*T1
    T4 = test_x['Premium_sum']*T2
    A=1
    return T3,T4,A

def train_model3(train_x,train_y,test_x,Mask1,Mask2):
    rgb_model.fit(train_x[Mask1],train_y[Mask1])
    T1,T2 = rgb_model.predict(train_x),rgb_model.predict(test_x)
    T1[T1<0]=0
    T2[T2<0]=0
    A = try_par(train_y[Mask2],T1[Mask2])
    T1,T2 = T1*A,T2*A
    return T1,T2,A

def and2(A,B):
    C=A.copy()
    C[C]=B
    return C
def div1(Y,N1,N2):
    Z = Y.copy()
    A = Y.copy()
    Z[A>=N1] = 0
    Z[np.logical_and(A<N1,A>N2)] = 1
    Z[A<=N2] = 2
    return Z
def div_count(N):
    L=int(N.max())+1
    S = [0]*L
    for i in range(L):
        S[i] = (N==i).sum()
    return S

In [2]:
train = pd.read_csv('../data/training_policy_c2.csv')
T = pd.read_csv('../data/duplicate_policy.csv',header=None)
T = T[0]
TF = train['Policy_Number']!=T[0]
for i in range(1,len(T)):
    TF = np.logical_and(TF,train['Policy_Number']!=T[i])
train = train[TF]
train = train.reset_index(drop=True)

In [3]:
test = pd.read_csv('../data/testing_policy_c2.csv')
train_y = train['Next_Premium'].copy()
train_y2 = (train_y/train['Premium_sum'])
del train['Next_Premium']
del test['Next_Premium']
test_PN = test['Policy_Number']
del train['Policy_Number']
del test['Policy_Number']
train_x = train
test_x = test

In [4]:
N1,N2= 2000, 0
d00 = div1(train_y,N1,N2)
S00 = div_count(d00)

P00,P01 = train_model(train_x,d00,test_x,train_x)

Mask000,Mask001,Mask002 = P00==0,P00==1,P00==2
Mask010,Mask011,Mask012 = P01==0,P01==1,P01==2

Mask1 = train_y>1500
Mask2 = np.logical_and(train_y<=2000,train_y>0)

T1,T2,A1 = train_model3(train_x,train_y,test_x,Mask1,Mask000)
T3,T4,A2 = train_model3(train_x,train_y,test_x,Mask2,Mask001)

A00,A01 = train_x['Premium_sum'] * 1,test_x['Premium_sum']*1
A00[Mask002]=0
A01[Mask012]=0

mae1 = mean_absolute_error(train_y,A00)
print(A1,A2,mae1)

0.967 0.986 2140.677918932396


In [6]:
N1,N2= 5000,2500
d10 = div1(train_y[P00!=2],N1,N2)
S10 = div_count(d10)

P10,P11 = train_model(train_x[P00!=2],d10,test_x[P01!=2],train_x[P00!=2])

Mask100,Mask101,Mask102 = and2(P00!=2,P10==0),and2(P00!=2,P10==1),and2(P00!=2,P10==2)
Mask110,Mask111,Mask112 = and2(P01!=2,P11==0),and2(P01!=2,P11==1),and2(P01!=2,P11==2)

Mask3 = np.logical_and(train_y<=2600,train_y>0)
T5,T6,A3 = train_model3(train_x,train_y,test_x,Mask3,Mask102)

A10,A11 = A00.copy(),A01.copy()
A10[Mask102]=T5[Mask102]
A11[Mask112]=T6[Mask112]

mae1 = mean_absolute_error(train_y,A10)
print(A1,A2,A3,mae1)

0.967 0.986 0.981 2111.5150640468537


In [7]:
N1,N2= 0.9,0.5
d20 = div1(train_y2[Mask100],N1,N2)
S20 = div_count(d20)

P20,P21 = train_model(train_x[Mask100],d20,test_x[Mask110],train_x[Mask100])
Mask200,Mask201,Mask202 = and2(Mask100,P20==0),and2(Mask100,P20==1),and2(Mask100,P20==2)
Mask210,Mask211,Mask212 = and2(Mask110,P21==0),and2(Mask110,P21==1),and2(Mask110,P21==2)

Mask1 = np.logical_and(train_y2>0.5,train_y2<1.25)
Mask2 = np.logical_and(train_y2>0.35,train_y2<0.95)
Mask3 = np.logical_and(train_y2>0.35,train_y2<0.75)
T1,T2,A1 = train_model2(train_x,train_y,train_y2,test_x,Mask1,Mask200)
T3,T4,A2 = train_model2(train_x,train_y,train_y2,test_x,Mask2,Mask201)
T5,T6,A3 = train_model2_1(train_x,train_y,train_y2,test_x,Mask3,Mask202)

A20,A21 = A10.copy(),A11.copy()
A20[Mask200]=T1[Mask200]
A20[Mask201]=T3[Mask201]
A20[Mask202]=T5[Mask202]
A21[Mask210]=T2[Mask210]
A21[Mask211]=T4[Mask211]
A21[Mask212]=T6[Mask212]

mae1 = mean_absolute_error(train_y,A20)
print(A1,A2,A3,mae1)

0.999 0.996 1 1568.047786030844


In [9]:
N1,N2= 0.95,0.5
d30 = div1(train_y2[Mask101],N1,N2)
S30 = div_count(d30)
P30,P31 = train_model(train_x[Mask101],d30,test_x[Mask111],train_x[Mask101])

Mask300,Mask301,Mask302 = and2(Mask101,P30==0),and2(Mask101,P30==1),and2(Mask101,P30==2)
Mask310,Mask311,Mask312 = and2(Mask111,P31==0),and2(Mask111,P31==1),and2(Mask111,P31==2)

Mask1 = np.logical_and(train_y2>0.45,train_y2<1.2)
Mask2 = np.logical_and(train_y2>0.35,train_y2<1)
Mask3 = np.logical_and(train_y2>0.4,train_y2<0.95)
T1,T2,A1 = train_model2(train_x,train_y,train_y2,test_x,Mask1,Mask300)
T3,T4,A2 = train_model2(train_x,train_y,train_y2,test_x,Mask2,Mask301)
T5,T6,A3 = train_model2_1(train_x,train_y,train_y2,test_x,Mask3,Mask302)

A30,A31 = A20.copy(),A21.copy()
A30[Mask300]=T1[Mask300]
A30[Mask301]=T3[Mask301]
A30[Mask302]=T5[Mask302]

A31[Mask310]=T2[Mask310]
A31[Mask311]=T4[Mask311]
A31[Mask312]=T6[Mask312]

mae1 = mean_absolute_error(train_y,A30)
print(A1,A2,A3,mae1)

0.994 0.997 1 1542.8940282731164


In [10]:
N1,N2= 0.95,0.5
d40 = div1(train_y2[Mask102],N1,N2)
S40 = div_count(d40)
P40,P41 = train_model(train_x[Mask102],d40,test_x[Mask112],train_x[Mask102])

Mask400,Mask401,Mask402 = and2(Mask102,P40==0),and2(Mask102,P40==1),and2(Mask102,P40==2)
Mask410,Mask411,Mask412 = and2(Mask112,P41==0),and2(Mask112,P41==1),and2(Mask112,P41==2)

Mask1 = np.logical_and(train_y2>0.05,train_y2<1.1)
Mask2 = np.logical_and(train_y2>0.5,train_y2<1)
T1,T2,A1 = train_model2(train_x,train_y,train_y2,test_x,Mask1,Mask400)
T3,T4,A2 = train_model2(train_x,train_y,train_y2,test_x,Mask2,Mask401)

A40,A41 = A30.copy(),A31.copy()
A40[Mask400]=T1[Mask400]
A40[Mask401]=T3[Mask401]
A41[Mask410]=T2[Mask410]
A41[Mask411]=T4[Mask411]

mae1 = mean_absolute_error(train_y,A40)
print(A1,A2,mae1)

1.002 0.987 1542.4068310158032


In [ ]:
output_csv = pd.DataFrame({"Policy_Number":test_PN ,"Next_Premium":A41})
output_csv.to_csv("10.csv", sep=',', index=False, header=True, encoding='utf_8_sig')